# Assignment3 Text mining
## Dataset Chinese text dataset ：
####       數據檔中包含10同類型的目錄，每個目錄裡面有一個代表的新聞主題，如"0體育"，"1房屋"，"2社會"，"3星座"等等，在不同主題中各有1000個文本資料，而我的學號後四碼為5422，因此我選擇："5娛樂"，"4科技"，"2社會"，進行我這次的文字探勘。

#### 匯入需要的套件&在開放資源找到的4種停字詞表

In [1]:
import random
import jieba
import pandas as pd
import matplotlib.pyplot as plt

#加载停用词
stopwords1=pd.read_csv('中文停用词表.txt',index_col=False,quoting=3,sep="\t", names=['stopword'], encoding='utf-8')
stopwords2=pd.read_csv('四川大学机器智能实验室停用词库.txt',index_col=False,quoting=3,sep="\t", names=['stopword'], encoding='utf-8')
stopwords3=pd.read_csv('百度停用词表.txt',index_col=False,quoting=3,sep="\t", names=['stopword'], encoding='utf-8')
stopwords4=pd.read_csv('哈工大停用词表.txt',index_col=False,quoting=3,sep="\t", names=['stopword'], encoding='utf-8')
st=pd.concat([stopwords1,stopwords2,stopwords3,stopwords4])
stopwords=st.values  

#### 匯入OS模塊，運用來建立文件，刪除文件，查詢文件等。
#### 匯入三種所需文字文本："5娛樂"，"4科技"，"2社會"。

In [2]:
import os

path = '5娛樂/'
lines1 =[]
for filename in os.listdir(path):
    if os.path.isfile(path + filename) and filename.endswith(".txt"):
        with open(path + filename, "r", encoding= 'utf-8') as file:
            data = file.read()
            lines1.append(data                    

path = '4科技/'
lines2 =[]
for filename in os.listdir(path):
    if os.path.isfile(path + filename) and filename.endswith(".txt"):
        with open(path + filename, "r", encoding= 'utf-8') as file:
            data = file.read()
            lines2.append(data)

path = '2社會/'
lines3 =[]
for filename in os.listdir(path):
    if os.path.isfile(path + filename) and filename.endswith(".txt"):
        with open(path + filename, "r", encoding= 'utf-8') as file:
            data = file.read()
            lines3.append(data)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



#### 資料前處理

In [3]:
#定義分詞和打標簽函數preprocess_text
#參數content_lines即為上面轉換的list
#參數sentences是定義的空list，用來儲存打標簽之後的數據
#參數category 是類型標簽
def preprocess_text(content_lines, classlabel):
    sentences =[]
    categories = []
    for line in content_lines:
        try:
            segs=jieba.lcut(line)
            segs = [v for v in segs if not str(v).isdigit()]#去數字
                                                            #filter() 函数用於過濾序列，過濾掉不符合條件的元素，返回由符合条件元素组成的新列表。
            segs = list(filter(lambda x:x.strip(), segs))   #去左右空格
            segs = list(filter(lambda x:len(x)>1, segs)) #長度為1的字符
            segs = list(filter(lambda x:x not in stopwords, segs)) #去掉停用詞
            sentences.append(segs)
            categories.append(classlabel)
        except Exception:
            print(line)
            continue
    
    return sentences, categories

In [5]:
X1, y1 = preprocess_text(lines1, 0)  #把第一類的文章分成第 0 類
X2, y2 = preprocess_text(lines2, 1)  #把第二類的文章分成第 1 類
X3, y3 = preprocess_text(lines3, 2)
X = X1 + X2 + X3  #把類別加起來
y = y1 + y2 + y3 

## 抽取詞向量特徵

In [7]:
# gensim modules
import gensim
from gensim import utils
from gensim import corpora,models


dictionary = corpora.Dictionary(X)  #開始做字典
corpus = [dictionary.doc2bow(text) for text in X]  #轉換成詞袋

In [8]:
model = models.TfidfModel(corpus)  #將15篇文章建模  , 改成TFIDF VALUE
corpus = [text for text in model[corpus]]
X = gensim.matutils.corpus2dense(corpus, num_terms=len(dictionary.token2id)).T

In [9]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import numpy as np
enc = OneHotEncoder()
y = enc.fit_transform(np.array(y).reshape(-1,1)).toarray()

In [10]:
from sklearn.model_selection import train_test_split   #訓練 & 測試資料
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [11]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam,SGD

Using TensorFlow backend.


In [ ]:
#Build the model  #建造模型keras語法
#input_shape=(A,B)，可建構2D層，也能只運用來表是一維層：input_shape=(A,)

model = Sequential()
model.add(Dense(32, input_dim=(55116), activation='relu', name='fc1'))
model.add(Dropout(0.5))

model.add(Dense(64, input_dim=(55116), activation='relu', name='fc1'))
model.add(Dropout(0.5))

model.add(Dense(128, input_dim=(55116), activation='relu', name='fc1'))
model.add(Dropout(0.5))

model.add(Dense(256, input_dim=(55116), activation='relu', name='fc1'))
model.add(Dropout(0.5))

model.add(Dense(3, activation='softmax', name='output'))

# Adam optimizer with learning rate of 0.001
optimizer = Adam(lr=1e-4)  # optimizer = Adam(lr=0.001) 訓練參數
model.compile(optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

#('optimizer' - 目標函數)  ('metrics' - 準確性)

print('Neural Network Model Summary: ')
print(model.summary())  #會告訴你有幾個參數要預測


import time  # 想要知道要跑多久 
start_time = time.time()

# Train the model - 調整參數在這邊調整

train = model.fit(X_train, y_train, verbose=2, batch_size=10, epochs=30, validation_split=0.1)

# epochs 跑幾圈
# batch_size 更新權重單元
#validation_split 驗證資料 = 0.1 (抽10%)  (把訓練的資料再抽一點出來 
#verbose=2 要不要看過程 -0(不看detail)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

print("--- %s seconds ---" % (time.time() - start_time))

# Test on unseen data
results = model.evaluate(X_test, y_test)

print('Final test set loss: {:4f}'.format(results[0]))  # loss 
print('Final test set accuracy: {:4f}'.format(results[1])) # accuracy

Neural Network Model Summary: 
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
fc1 (Dense)                  (None, 128)               7054976   
_________________________________________________________________
output (Dense)               (None, 3)                 387       
Total params: 7,055,363
Trainable params: 7,055,363
Non-trainable params: 0
_________________________________________________________________
None
Train on 2025 samples, validate on 225 samples
Epoch 1/30
 - 212s - loss: 0.8768 - accuracy: 0.5768 - val_loss: 0.6637 - val_accuracy: 0.5733
Epoch 2/30
 - 212s - loss: 0.5562 - accuracy: 0.6632 - val_loss: 0.5615 - val_accuracy: 0.4444
Epoch 3/30
 - 215s - loss: 0.4848 - accuracy: 0.6904 - val_loss: 0.5622 - val_accuracy: 0.4356
Epoch 4/30
 - 217s - loss: 0.4587 - accuracy: 0.6923 - val_loss: 0.5924 - val_accuracy: 0.4311
Epoch 5/30
 - 245s - loss: 0.4406 - accurac